In [36]:
import pandas as pd
import numpy as np
import pickle

import math
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from collections import Counter
from itertools import chain
from scipy.optimize import curve_fit
import powerlaw

from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers_all-MiniLM-L6-v2')
import oracledb
connection = oracledb.connect(user="", password="", host="",port=,service_name="")

import oracledb
from sqlalchemy import create_engine
import cx_Oracle
from tqdm import tqdm

from sentence_transformers.util import cos_sim
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from powerlaw import Fit, plot_pdf

from tqdm import tqdm
import re
import os
from keybert import KeyBERT
kw_model = KeyBERT(model=model)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve,precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# 下载NLTK的punkt数据
nltk.download('punkt')
stemmer = PorterStemmer()

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

def get_dict(x,dictionary):
    return [dictionary[str(item)] for item in x]
def get_larger(x,number):
    return [item for item in x if item>number]
def compare_larger(x,number):
    return any([item>number for item in x])
def convert_date(x):
    try:
        date=pd.Timestamp(x)
    except:
        date=pd.Timestamp(x.split('-')[0]+'-01-01')
    return date

[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>


In [37]:
def cosine_similarity(list1,list2):
    # Convert lists to numpy arrays
    vector1 = np.array(list1)
    vector2 = np.array(list2)

    # Calculate dot product
    dot_product = np.dot(vector1, vector2)

    # Calculate norms
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    # Calculate cosine similarity
    cosine_similarity = dot_product / (norm1 * norm2)

    return cosine_similarity

In [38]:
import itertools
def select_elements(x):
    # 创建一个空列表，用于存储满足条件的元素
    selected_elements = []

    # 获取DataFrame的行数和列数
    n_rows, n_cols = x.shape

    # 遍历每个元素，检查索引是否满足条件
    for i in range(n_rows):
        for j in range(n_cols):
            if j < i:
                selected_elements.append(x.iloc[i, j])
    return selected_elements

In [39]:
sql_string='select * from "biotechnology_cpc"'
df_biotechnology_cpc = pd.read_sql(sql=sql_string, con=connection)
df_biotechnology_cpc

/tmp/ipykernel_74757/2073867974.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_biotechnology_cpc = pd.read_sql(sql=sql_string, con=connection)


,appln_id,cpc_class_symbol
0,2,B01J2219/00718
1,2,C07K 16/005
2,2,C07K 16/18
3,2,C07K 16/241
4,2,C07K 16/26
...,...,...
72735225,575414791,A61B 1/00
72735226,575414791,A61B 1/00096
72735227,575414791,G02B 13/04
72735228,575414791,G02B 23/243


In [40]:
sql_string='select t."appln_id", t."appln_filing_year" from "biotechnology_basic_info" t'
biotechnology_basic_info = pd.read_sql(sql=sql_string, con=connection)
biotechnology_basic_info

/tmp/ipykernel_74757/3122283556.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  biotechnology_basic_info = pd.read_sql(sql=sql_string, con=connection)


,appln_id,appln_filing_year
0,72,2004
1,79,2005
2,88,2005
3,116,2006
4,119,2006
...,...,...
10676891,575014548,2021
10676892,575014578,2021
10676893,575014590,2021
10676894,575014598,2021


In [41]:
df_biotechnology_cpc=df_biotechnology_cpc.merge(biotechnology_basic_info,how='left',left_on='appln_id',right_on='appln_id')
df_biotechnology_cpc['cpc_class_symbol_4']=df_biotechnology_cpc['cpc_class_symbol'].apply(lambda x:x[0:4])
df_biotechnology_cpc

,appln_id,cpc_class_symbol,appln_filing_year,cpc_class_symbol_4
0,2,B01J2219/00718,1992,B01J
1,2,C07K 16/005,1992,C07K
2,2,C07K 16/18,1992,C07K
3,2,C07K 16/241,1992,C07K
4,2,C07K 16/26,1992,C07K
...,...,...,...,...
72735225,575414791,A61B 1/00,2022,A61B
72735226,575414791,A61B 1/00096,2022,A61B
72735227,575414791,G02B 13/04,2022,G02B
72735228,575414791,G02B 23/243,2022,G02B


In [42]:
filing_year_list=list(range(1995,2023))
dict_year_pairs={}
for filing_year in filing_year_list:
    print(filing_year,'----------------------------')
    df_item=df_biotechnology_cpc[df_biotechnology_cpc['appln_filing_year']==filing_year]
    pair_list=[]
    total_cpc_list=df_item['cpc_class_symbol_4'].unique()
    for appln_id in tqdm(df_item['appln_id'].unique()):
        cpc_4_list=df_item[df_item['appln_id']==appln_id]['cpc_class_symbol_4'].unique()
        cpc_4_pair=list(itertools.combinations(cpc_4_list, 2))
        pair_list.append(cpc_4_pair)
    year_pairs=list(chain(*pair_list))
    dict_year_pairs[filing_year]=year_pairs

1995 ----------------------------


100%|█████████████████████████████████████████████████████████████████████████| 123432/123432 [01:36<00:00, 1281.18it/s]


1996 ----------------------------


100%|█████████████████████████████████████████████████████████████████████████| 127826/127826 [01:42<00:00, 1245.68it/s]


1997 ----------------------------


...

100%|██████████████████████████████████████████████████████████████████████████| 366784/366784 [13:19<00:00, 459.03it/s]


2020 ----------------------------


100%|██████████████████████████████████████████████████████████████████████████| 352257/352257 [11:06<00:00, 528.17it/s]


2021 ----------------------------


 49%|███████████████████████████████████▉                                      | 112714/232221 [02:21<02:30, 794.68it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
len(total_cpc_list)

57238

In [43]:
filing_year_list=list(range(2000,2023))
for filing_year in tqdm(filing_year_list):
    print(filing_year,'----------------------------')
    df_item=df_biotechnology_cpc[df_biotechnology_cpc['appln_filing_year'].isin(range(filing_year-0,filing_year+1))]
    total_cpc_list=df_item['cpc_class_symbol_4'].unique()
    df_matrix=pd.DataFrame(columns=total_cpc_list,index=total_cpc_list)
    df_sub=pd.DataFrame(columns=total_cpc_list,index=total_cpc_list)
    
    dict_pair_count=Counter(chain(*[dict_year_pairs[year] for year in range(filing_year-0,filing_year+1)]))
    for k in dict_pair_count.keys():
        df_matrix.loc[k[0],k[1]]=dict_pair_count[k]
        df_matrix.loc[k[1],k[0]]=dict_pair_count[k]
    df_matrix.fillna(0,inplace=True)
    df_matrix.to_pickle('matrix/{}_matrix_1.pickle'.format(str(filing_year)))
    selected_elements=select_elements(df_matrix)

    mean=np.mean(selected_elements)
    std=np.std(selected_elements)
    df_matrix_st=(df_matrix-mean)/std

    selected_elements_st=select_elements(df_matrix_st)
    min_value=np.min(selected_elements_st)
    max_value=np.max(selected_elements_st)
    df_matrix_mm=(df_matrix_st-min_value)/ (max_value - min_value)
    df_matrix_mm.to_pickle('matrix/{}_matrix_mm_1.pickle'.format(str(filing_year)))
    
    # 获取DataFrame的行数和列数
    n_rows, n_cols = df_sub.shape

    # 遍历每个元素，检查索引是否满足条件
    for i in range(n_rows):
        for j in range(n_cols):
            if j < i:
                cosine_similarity_i_j=cosine_similarity(df_matrix_mm.iloc[:, i],df_matrix_mm.iloc[:, j])
                df_sub.iloc[i,j]=cosine_similarity_i_j
                df_sub.iloc[j,i]=cosine_similarity_i_j
    df_sub.to_pickle('matrix/{}_matrix_sub_1.pickle'.format(str(filing_year)))


  0%|                                                                                            | 0/23 [00:00<?, ?it/s]

2000 ----------------------------


/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
  4%|███▋                                                                                | 1/23 [00:27<10:11, 27.80s/it]

2001 ----------------------------


/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
  9%|███████▎                                                                            | 2/23 [00:55<09:47, 27.96s/it]

...

/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
 91%|███████████████████████████████████████████████████████████████████████████▊       | 21/23 [11:08<01:09, 34.91s/it]

2021 ----------------------------



 96%|███████████████████████████████████████████████████████████████████████████████▍   | 22/23 [11:41<00:34, 34.19s/it]

2022 ----------------------------


/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
100%|███████████████████████████████████████████████████████████████████████████████████| 23/23 [11:55<00:00, 31.13s/it]


In [44]:
filing_year_list=list(range(2000,2023))
for filing_year in tqdm(filing_year_list):
    print(filing_year,'----------------------------')
    df_item=df_biotechnology_cpc[df_biotechnology_cpc['appln_filing_year'].isin(range(filing_year-2,filing_year+1))]
    total_cpc_list=df_item['cpc_class_symbol_4'].unique()
    df_matrix=pd.DataFrame(columns=total_cpc_list,index=total_cpc_list)
    df_sub=pd.DataFrame(columns=total_cpc_list,index=total_cpc_list)
    
    dict_pair_count=Counter(chain(*[dict_year_pairs[year] for year in range(filing_year-2,filing_year+1)]))
    for k in dict_pair_count.keys():
        df_matrix.loc[k[0],k[1]]=dict_pair_count[k]
        df_matrix.loc[k[1],k[0]]=dict_pair_count[k]
    df_matrix.fillna(0,inplace=True)
    df_matrix.to_pickle('matrix/{}_matrix_3.pickle'.format(str(filing_year)))
    selected_elements=select_elements(df_matrix)

    mean=np.mean(selected_elements)
    std=np.std(selected_elements)
    df_matrix_st=(df_matrix-mean)/std

    selected_elements_st=select_elements(df_matrix_st)
    min_value=np.min(selected_elements_st)
    max_value=np.max(selected_elements_st)
    df_matrix_mm=(df_matrix_st-min_value)/ (max_value - min_value)
    df_matrix_mm.to_pickle('matrix/{}_matrix_mm_3.pickle'.format(str(filing_year)))
    
    # 获取DataFrame的行数和列数
    n_rows, n_cols = df_sub.shape

    # 遍历每个元素，检查索引是否满足条件
    for i in range(n_rows):
        for j in range(n_cols):
            if j < i:
                cosine_similarity_i_j=cosine_similarity(df_matrix_mm.iloc[:, i],df_matrix_mm.iloc[:, j])
                df_sub.iloc[i,j]=cosine_similarity_i_j
                df_sub.iloc[j,i]=cosine_similarity_i_j
    df_sub.to_pickle('matrix/{}_matrix_sub_3.pickle'.format(str(filing_year)))


  0%|                                                                                            | 0/23 [00:00<?, ?it/s]

2000 ----------------------------


/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
  4%|███▋                                                                                | 1/23 [00:33<12:16, 33.47s/it]

2001 ----------------------------


/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
  9%|███████▎                                                                            | 2/23 [01:07<11:55, 34.06s/it]

...


 91%|███████████████████████████████████████████████████████████████████████████▊       | 21/23 [13:15<01:21, 40.59s/it]

2021 ----------------------------



 96%|███████████████████████████████████████████████████████████████████████████████▍   | 22/23 [13:54<00:40, 40.31s/it]

2022 ----------------------------


/tmp/ipykernel_74757/3711813477.py:14: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = dot_product / (norm1 * norm2)
100%|███████████████████████████████████████████████████████████████████████████████████| 23/23 [14:32<00:00, 37.95s/it]


In [20]:
df_matrix_mm

,A61B,A61K,A61N,A61P,G01T,A61F,A61M,C07K,G01R,H01Q,...,F23M,B27F,F16T,G06J,C21C,B61H,B42B,G99Z,G09D,F41F
A61B,509.119206,0.007905,0.184524,0.002932,0.002449,0.253368,0.264056,0.002115,0.012915,0.003043,...,0.0,0.000037,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
A61K,0.007905,461.703952,0.012618,0.809946,0.007348,0.004565,0.032919,0.519057,0.000779,0.002858,...,0.0,0.000000,0.0,0.000074,0.0,0.000000,0.0,0.0,0.0,0.0
A61N,0.184524,0.012618,23.693524,0.000854,0.000186,0.009389,0.025051,0.000816,0.000037,0.080163,...,0.0,0.000000,0.0,0.000074,0.0,0.000000,0.0,0.0,0.0,0.0
A61P,0.002932,0.809946,0.000854,123.562665,0.001522,0.002227,0.062015,0.280794,0.004268,0.001967,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
G01T,0.002449,0.007348,0.000186,0.001522,0.224569,0.000779,0.013657,0.000371,0.006940,0.000891,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B61H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000037,0.0,0.0,0.0,0.0
B42B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
G99Z,0.000000,0.000000,0.000000,0.000000,0.000000,0.000037,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
G09D,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
